In [59]:
import numpy as np
import pandas as pd

from sklearn.impute import (SimpleImputer,KNNImputer)
from sklearn.ensemble import (RandomForestRegressor, IsolationForest)
from sklearn.neighbors import LocalOutlierFactor

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import lightgbm as lgb

from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [60]:
def fill_missing_values(X, n_neighbors = 75, method="KNN"): 
    
    # normalization
    X_std = np.nanstd(X,axis=0,keepdims=True)
    X_ave = np.nanmean(X,axis=0,keepdims=True)
    X_norma = (X-X_ave)/X_std
    
    # use KNNImputer
    imputer = KNNImputer(missing_values=np.nan, n_neighbors=n_neighbors, weights = 'distance') if method=="KNN"\
        else SimpleImputer(missing_values=np.nan, strategy='median')
    
    X_norma_fixed = imputer.fit_transform(X_norma)
    
    return X_norma_fixed

def remove_outliers(X, y):
    print("IsolationForest-Traing data shape before removed: {}".format(X.shape))
    Z = np.c_[X, y]
    iforest = IsolationForest(max_samples=200, random_state=1, contamination=0.005)
    iforest.fit(Z)
    iforest_outlier_pred = iforest.predict(Z)
    
    Z = np.c_[X, y]
    local = LocalOutlierFactor(n_neighbors=150, contamination=0.005)
    local.fit(Z)
    local_outlier_pred = local.fit_predict(Z)

    mask = np.logical_and((iforest_outlier_pred!=-1), (local_outlier_pred!=-1))
    X , y = X[mask, :], y[mask]
    print("IsolationForest-Traing data shape after removed: {}".format(X.shape))
    return X, y

def select_features(X, y, X_test, feature_num=50):
    rf = RandomForestRegressor(n_jobs=-1, n_estimators=80, random_state=1)
    rf.fit(X, y)
    indices = np.asarray(list(rf.feature_importances_)).argsort()[-feature_num:][::-1]
    
    X = np.take(X, indices, axis = 1)
    X_test = np.take(X_test, indices, axis = 1)
    return X, X_test

In [61]:
X_train_data = pd.read_csv('X_train.csv')
y_train_data = pd.read_csv('y_train.csv')
X_test_data = pd.read_csv('X_test.csv')

indices_test = np.array(X_test_data)[:,0]
X_test = np.array(X_test_data)[:,1:]
y_train = np.array(y_train_data)[:,1]
X_train = np.array(X_train_data)[:,1:]

## 1. Imputation of Missing Values
* [Reference](https://scikit-learn.org/stable/modules/impute.html)
* We use median of column instead of mean

In [62]:
X_train_missing_indices = X_train[X_train==np]
X_train = fill_missing_values(X_train, n_neighbors=75)
X_test_processed = fill_missing_values(X_test)
print(X_train.shape)
print(X_test_processed.shape)

(1212, 828)
(776, 828)


## 2. Outlier Detection
* [reference_sklearn](https://scikit-learn.org/stable/modules/outlier_detection.html)
* [reference_in_detail](https://practicaldatascience.co.uk/machine-learning/how-to-use-the-isolation-forest-model-for-outlier-detection)

In [63]:
X_processed,y_processed = remove_outliers(X_train,y_train)

IsolationForest-Traing data shape before removed: (1212, 828)
IsolationForest-Traing data shape after removed: (1200, 828)


## 3. Feature Selection

In [64]:
# pearsonr feature selection
from scipy.stats import pearsonr
from sklearn.decomposition import PCA

def pearsonr_feature_selection(X, y, X_test,degree,weights):
    feature_idx =  np.array([i for i in range(0,X.shape[1])])
    correlations = {}
#     print("feature_idx: ", feature_idx.T)
    for idx in feature_idx:
        correlation = 0
        for degree_idx in range(0,len(weights)):
            correlation += abs(weights[degree_idx]*pearsonr(X[:,idx],y**degree[degree_idx])[0])
        correlations[idx] = correlation
    data_correlations = pd.DataFrame(correlations, index=['correlation']).T
    indices_desc = data_correlations['correlation'].abs().sort_values(ascending=False).index
#     print(indices_desc)
    return data_correlations,indices_desc

def pearsonr_feature_selection_correlation(X, X_test,threshold):
    feature_idx =  np.array([i for i in range(0,X.shape[1])])
    correlation_select = np.array([True for i in range(0,X.shape[1])])
    for f_idx in feature_idx:
        if(correlation_select[f_idx]):
            feature_co,indices_desc = pearsonr_feature_selection(X[:,(f_idx+1):], X[:,f_idx], X_test_processed,np.array([1]),np.array([1]))
            reomove_features = feature_co.loc[indices_desc][abs(feature_co['correlation']) >= threshold]
            remove_index = reomove_features.index+(f_idx+1)
            correlation_select[remove_index] = False
    
    return X[:,correlation_select],X_test[:,correlation_select]

In [73]:
data_relation,indices_desc = pearsonr_feature_selection(X_processed, y_processed, X_test_processed,np.array([1,-3,3]),np.array([0.9,0.05,0.05]))

In [110]:
selected_features = data_relation.loc[indices_desc][abs(data_relation['correlation']) >= 0.025]
X_selection = X_processed[:,selected_features.index]
X_test_selection = X_test_processed[:,selected_features.index]
print(X_selection.shape)
print(X_test_selection.shape)

(1200, 469)
(776, 469)


In [234]:
# X, X_test = feature_reduction(X, X_test,750)
X_selection_tree, X_test_selection_tree = select_features(X_selection, y_processed, X_test_selection,feature_num = 105)
print("Traing data shape after selection: {}".format(X_selection_tree.shape))
print("Testing data shape after selection: {}".format(X_test_selection_tree.shape))

Traing data shape after selection: (1200, 105)
Testing data shape after selection: (776, 105)


In [235]:
X_selection_tree_cor, X_test_selection_tree_cor = pearsonr_feature_selection_correlation(X_selection_tree, X_test_selection_tree,0.93)
# X_selection_tree_cor, X_test_selection_tree_cor = X_selection_tree, X_test_selection_tree
print(X_selection_tree_cor.shape)
print(X_test_selection_tree_cor.shape)

(1200, 98)
(776, 98)


## 4. Extra Tree

In [230]:
def custom_r2(prediction, train_data):
    """Regular r2 cost function returned as a tuple to be used with lgb"""
    labels = train_data.get_label()
    return 'r2', r2_score(labels, prediction), True

def fit_model_and_pred(X_train, y_train, X_val, y_val, X_test):
   
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'num_leaves': 1800,
        'learning_rate': 0.025,
        'max_depth': 11,
        'n_estimators': 1000,
        'min_child_weight': 1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'feature_fraction': 0.5,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0,
        'num_iterations':1000,
    }
    
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=500,
                    feval=custom_r2,
                    valid_sets={lgb_train, lgb_eval},
                    early_stopping_rounds=100,
                    verbose_eval=True
                   )
    
    y_val_pred = gbm.predict(X_val)
    score = r2_score(y_val, y_val_pred)
    y_pred = gbm.predict(X_test) 
    
    return score, y_pred

def train_k_fold(X,X_test, y, fold_num=10):
    kf = KFold(n_splits=fold_num, random_state=None, shuffle=False)
    kf.get_n_splits(X)
    test_score = 0.0
    y_test_predict = np.zeros(X_test.shape[0])
    for train_index, test_index in kf.split(X):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]

        score, y_pred = fit_model_and_pred(X_train, y_train, X_val, y_val, X_test)
        y_test_predict += y_pred
        print('The obtained validation r2 score is : ',score)
        test_score += score
    print("Validation score: %f"%(test_score/fold_num))
    return y_test_predict/fold_num

def train_k_fold_predict(X, y, X_test, fold_num=10):
    kf = KFold(n_splits=fold_num)
    kf.get_n_splits(X)
    y_test_predict = np.zeros(X_test.shape[0])
    cnt = 0
    val_score = 0.0
        
    for train_index, test_index in kf.split(X):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]

        score, y_pred = fit_model_and_pred(X_train, y_train, X_val, y_val, X_test)
        val_score += score
        if(score > 0.5):
            y_test_predict += y_pred
            cnt += 1
    return val_score/fold_num, y_test_predict/cnt

In [231]:
y_test_pred = train_k_fold(X_selection_tree_cor,X_test_selection_tree_cor, y_processed, fold_num=10) 

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001917 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	training's l2: 93.1297	training's

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[76]	training's l2: 20.3077	training's r2: 0.787991	valid_0's l2: 30.4104	valid_0's r2: 0.641892
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[77]	training's l2: 20.0393	training's r2: 0.790793	valid_0's l2: 30.3507	valid_0's r2: 0.642595
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[78]	training's l2: 19.7696	training's r2: 0.793609	valid_0's l2: 30.2774	valid_0's r2: 0.643458
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[79]	training's l2: 19.4962	training's r2: 0.796463	valid_0's l2: 30.1622	valid_0's r2: 0.644815
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[80]	training's l2: 19.2417	training's r2: 0.79912	valid_0's l2: 29.9532	valid_0's r2: 0.647277
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[81]	training's l2: 19.0058	training's r2: 0.801583	valid_0's l2: 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[139]	training's l2: 10.3526	training's r2: 0.891921	valid_0's l2: 26.0744	valid_0's r2: 0.692952
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[140]	training's l2: 10.256	training's r2: 0.892929	valid_0's l2: 26.0079	valid_0's r2: 0.693735
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[141]	training's l2: 10.1633	training's r2: 0.893897	valid_0's l2: 25.9515	valid_0's r2: 0.6944
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[142]	training's l2: 10.1013	training's r2: 0.894544	valid_0's l2: 25.9422	valid_0's r2: 0.69451
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[143]	training's l2: 10.0213	training's r2: 0.895379	valid_0's l2: 25.8758	valid_0's r2: 0.695291
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[144]	training's l2: 9.94363	training's r2: 0.89619	valid_0's l2

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[221]	training's l2: 6.00801	training's r2: 0.937277	valid_0's l2: 24.5541	valid_0's r2: 0.710855
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[222]	training's l2: 5.96845	training's r2: 0.93769	valid_0's l2: 24.5289	valid_0's r2: 0.711152
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[223]	training's l2: 5.92653	training's r2: 0.938128	valid_0's l2: 24.565	valid_0's r2: 0.710726
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[224]	training's l2: 5.8905	training's r2: 0.938504	valid_0's l2: 24.6059	valid_0's r2: 0.710245
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[225]	training's l2: 5.85455	training's r2: 0.93888	valid_0's l2: 24.6092	valid_0's r2: 0.710207
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[226]	training's l2: 5.82504	training's r2: 0.939188	valid_0's l

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[306]	training's l2: 3.92263	training's r2: 0.959048	valid_0's l2: 24.182	valid_0's r2: 0.715236
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[307]	training's l2: 3.90748	training's r2: 0.959207	valid_0's l2: 24.1892	valid_0's r2: 0.715152
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[308]	training's l2: 3.88446	training's r2: 0.959447	valid_0's l2: 24.1888	valid_0's r2: 0.715157
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[309]	training's l2: 3.86517	training's r2: 0.959648	valid_0's l2: 24.1708	valid_0's r2: 0.715369
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[310]	training's l2: 3.84137	training's r2: 0.959897	valid_0's l2: 24.1452	valid_0's r2: 0.715671
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[311]	training's l2: 3.82292	training's r2: 0.960089	valid_0'

...

[743]	training's l2: 0.719444	training's r2: 0.992408	valid_0's l2: 41.9354	valid_0's r2: 0.56191
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[744]	training's l2: 0.717986	training's r2: 0.992423	valid_0's l2: 41.9215	valid_0's r2: 0.562056
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[745]	training's l2: 0.716167	training's r2: 0.992442	valid_0's l2: 41.9281	valid_0's r2: 0.561987
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[746]	training's l2: 0.713914	training's r2: 0.992466	valid_0's l2: 41.9211	valid_0's r2: 0.56206
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[747]	training's l2: 0.712222	training's r2: 0.992484	valid_0's l2: 41.9214	valid_0's r2: 0.562056
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[748]	training's l2: 0.708652	training's r2: 0.992521	valid_0's l2: 41.9252	valid_0's r2: 0.562017
[LightGBM] [Warning] No further s

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[807]	training's l2: 0.603625	training's r2: 0.99363	valid_0's l2: 41.9327	valid_0's r2: 0.561938
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[808]	training's l2: 0.602483	training's r2: 0.993642	valid_0's l2: 41.9202	valid_0's r2: 0.562069
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[809]	training's l2: 0.601536	training's r2: 0.993652	valid_0's l2: 41.8968	valid_0's r2: 0.562314
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[810]	training's l2: 0.60019	training's r2: 0.993666	valid_0's l2: 41.8825	valid_0's r2: 0.562463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[811]	training's l2: 0.598156	training's r2: 0.993688	valid_0's l2: 41.8889	valid_0's r2: 0.562396
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[812]	training's l2: 0.596429	training's r2: 0.993706	val

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[868]	training's l2: 0.514757	training's r2: 0.994568	valid_0's l2: 41.7239	valid_0's r2: 0.56412
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[869]	training's l2: 0.513668	training's r2: 0.994579	valid_0's l2: 41.7284	valid_0's r2: 0.564073
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[870]	training's l2: 0.512871	training's r2: 0.994588	valid_0's l2: 41.7165	valid_0's r2: 0.564197
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[871]	training's l2: 0.51179	training's r2: 0.994599	valid_0's l2: 41.7206	valid_0's r2: 0.564154
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[872]	training's l2: 0.510338	training's r2: 0.994614	valid_0's l2: 41.7111	valid_0's r2: 0.564254
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[873]	training's l2: 0.508642	training's r2: 0.994632	val

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[936]	training's l2: 0.440732	training's r2: 0.995349	valid_0's l2: 41.6496	valid_0's r2: 0.564896
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[937]	training's l2: 0.439963	training's r2: 0.995357	valid_0's l2: 41.6453	valid_0's r2: 0.564941
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[938]	training's l2: 0.439031	training's r2: 0.995367	valid_0's l2: 41.6434	valid_0's r2: 0.564961
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[939]	training's l2: 0.438316	training's r2: 0.995374	valid_0's l2: 41.6403	valid_0's r2: 0.564994
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[940]	training's l2: 0.436636	training's r2: 0.995392	valid_0's l2: 41.6469	valid_0's r2: 0.564924
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[941]	training's l2: 0.435058	training's r2: 0.995409	v

[1000]	training's l2: 0.381412	training's r2: 0.995975	valid_0's l2: 41.6415	valid_0's r2: 0.564981
Did not meet early stopping. Best iteration is:
[1000]	training's l2: 0.381412	training's r2: 0.995975	valid_0's l2: 41.6415	valid_0's r2: 0.564981
The obtained validation r2 score is :  0.5649810003591685
Validation score: 0.645148


In [222]:
y_test_pred
final_res = np.vstack((indices_test, y_test_pred)).T

In [181]:
score, y_test_pred = train_k_fold_predict(X_selection_tree_cor, y_processed, X_test_selection_tree_cor)
final_res = np.vstack((indices_test, y_test_pred)).T
print("validation score: ", score)

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002391 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	training's l2: 92.9629	training's

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[73]	training's l2: 20.6359	training's r2: 0.784564	valid_1's l2: 30.3437	valid_1's r2: 0.642678
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[74]	training's l2: 20.3635	training's r2: 0.787408	valid_1's l2: 30.2018	valid_1's r2: 0.644349
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[75]	training's l2: 20.0871	training's r2: 0.790294	valid_1's l2: 29.9622	valid_1's r2: 0.647171
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[76]	training's l2: 19.8112	training's r2: 0.793175	valid_1's l2: 29.8805	valid_1's r2: 0.648132
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[77]	training's l2: 19.5316	training's r2: 0.796094	valid_1's l2: 29.8031	valid_1's r2: 0.649044
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[78]	training's l2: 19.2542	training's r2: 0.798989	valid_1's l2:

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[147]	training's l2: 9.04973	training's r2: 0.905522	valid_1's l2: 24.6832	valid_1's r2: 0.709335
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[148]	training's l2: 8.98613	training's r2: 0.906186	valid_1's l2: 24.6455	valid_1's r2: 0.709779
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[149]	training's l2: 8.90202	training's r2: 0.907064	valid_1's l2: 24.6153	valid_1's r2: 0.710134
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[150]	training's l2: 8.83727	training's r2: 0.90774	valid_1's l2: 24.5552	valid_1's r2: 0.710842
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[151]	training's l2: 8.76373	training's r2: 0.908508	valid_1's l2: 24.5028	valid_1's r2: 0.71146
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[152]	training's l2: 8.7011	training's r2: 0.909162	valid_1's 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[200]	training's l2: 6.07538	training's r2: 0.936574	valid_1's l2: 23.3967	valid_1's r2: 0.724484
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[201]	training's l2: 6.04013	training's r2: 0.936942	valid_1's l2: 23.3793	valid_1's r2: 0.724689
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[202]	training's l2: 6.0069	training's r2: 0.937289	valid_1's l2: 23.4106	valid_1's r2: 0.724321
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[203]	training's l2: 5.97977	training's r2: 0.937572	valid_1's l2: 23.4441	valid_1's r2: 0.723927
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[204]	training's l2: 5.9534	training's r2: 0.937848	valid_1's l2: 23.4967	valid_1's r2: 0.723307
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[205]	training's l2: 5.92951	training's r2: 0.938097	valid_1's

[258]	training's l2: 4.24968	training's r2: 0.955634	valid_1's l2: 23.123	valid_1's r2: 0.727708
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[259]	training's l2: 4.23485	training's r2: 0.955789	valid_1's l2: 23.107	valid_1's r2: 0.727896
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[260]	training's l2: 4.21963	training's r2: 0.955948	valid_1's l2: 23.0873	valid_1's r2: 0.728127
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[261]	training's l2: 4.19808	training's r2: 0.956173	valid_1's l2: 23.0888	valid_1's r2: 0.728111
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[262]	training's l2: 4.1792	training's r2: 0.95637	valid_1's l2: 23.0665	valid_1's r2: 0.728373
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[263]	training's l2: 4.15604	training's r2: 0.956612	valid_1's l2: 23.0545	valid_1's r2: 0.728514
[LightGBM] [Warning] No further splits wi

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[697]	training's l2: 0.62205	training's r2: 0.993435	valid_0's l2: 42.2269	valid_0's r2: 0.558865
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[698]	training's l2: 0.620221	training's r2: 0.993455	valid_0's l2: 42.2129	valid_0's r2: 0.559011
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[699]	training's l2: 0.618808	training's r2: 0.99347	valid_0's l2: 42.2112	valid_0's r2: 0.559029
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[700]	training's l2: 0.617252	training's r2: 0.993486	valid_0's l2: 42.2139	valid_0's r2: 0.559001
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[701]	training's l2: 0.615825	training's r2: 0.993501	valid_0's l2: 42.2112	valid_0's r2: 0.559029
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[702]	training's l2: 0.614881	training's r2: 0.993511	val

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[792]	training's l2: 0.480802	training's r2: 0.994926	valid_0's l2: 42.208	valid_0's r2: 0.559063
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[793]	training's l2: 0.479478	training's r2: 0.99494	valid_0's l2: 42.2153	valid_0's r2: 0.558986
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[794]	training's l2: 0.478006	training's r2: 0.994956	valid_0's l2: 42.2297	valid_0's r2: 0.558836
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[795]	training's l2: 0.477117	training's r2: 0.994965	valid_0's l2: 42.2278	valid_0's r2: 0.558856
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[796]	training's l2: 0.475824	training's r2: 0.994979	valid_0's l2: 42.2398	valid_0's r2: 0.558731
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[797]	training's l2: 0.47437	training's r2: 0.994994	vali

[864]	training's l2: 0.397608	training's r2: 0.995804	valid_0's l2: 42.1215	valid_0's r2: 0.559967
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[865]	training's l2: 0.397023	training's r2: 0.99581	valid_0's l2: 42.1177	valid_0's r2: 0.560006
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[866]	training's l2: 0.396349	training's r2: 0.995817	valid_0's l2: 42.1142	valid_0's r2: 0.560042
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[867]	training's l2: 0.395724	training's r2: 0.995824	valid_0's l2: 42.1121	valid_0's r2: 0.560064
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[868]	training's l2: 0.39498	training's r2: 0.995832	valid_0's l2: 42.1165	valid_0's r2: 0.560019
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[869]	training's l2: 0.393955	training's r2: 0.995843	valid_0's l2: 42.1152	valid_0's r2: 0.560032
[LightGBM] [Warning] No further s

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[947]	training's l2: 0.326392	training's r2: 0.996556	valid_0's l2: 42.1568	valid_0's r2: 0.559597
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[948]	training's l2: 0.325832	training's r2: 0.996561	valid_0's l2: 42.1481	valid_0's r2: 0.559688
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[949]	training's l2: 0.325136	training's r2: 0.996569	valid_0's l2: 42.1395	valid_0's r2: 0.559778
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[950]	training's l2: 0.324229	training's r2: 0.996578	valid_0's l2: 42.1332	valid_0's r2: 0.559844
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[951]	training's l2: 0.32336	training's r2: 0.996588	valid_0's l2: 42.1343	valid_0's r2: 0.559833
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[952]	training's l2: 0.32225	training's r2: 0.996599	val

validation score:  0.6324591648104292


In [223]:
df_res = pd.DataFrame(final_res)
df_res.to_csv("our_result.csv", header = ["id", "y"], index=False)